In [1]:
# Set input and output folders 
inputPath = "/data/students/bigdata-01QYD/Lab6_DBD/Reviews.csv"
outputPath = "Lab6/res_Lab_6"

In [2]:
# Read input data
listAmazonRDD = sc.textFile(inputPath)

# Task 1
- Transposes the original Amazon food dataset, obtaining an RDD of pairs (tuples) of the type: (user_id, list of the product_ids reviewed by user_id)
- Counts the frequencies of all the pairs of products reviewed together
- Stores on the output folder all the pairs of products that appear more than once and their frequencies. The pairs of products must be sorted by decreasing frequency. 

In [3]:
# Discard the header
listAmazonRDD_noHeader = listAmazonRDD.filter(lambda line : line.startswith("Id,")==False)

In [4]:
# Create new RDD
def extractUserID_ProductIDs(line):
    columns = line.split(",")
    userID = columns[2]
    productID = columns[1]
    
    return (userID, productID)

UserID_ProductID_RDD = listAmazonRDD_noHeader.map(extractUserID_ProductIDs)
UserID_ProductID_distinctRDD = UserID_ProductID_RDD.distinct()
UserID_ProductID_ProductListRDD = UserID_ProductID_distinctRDD.groupByKey()

In [5]:
ProductID_ValuesRDD = UserID_ProductID_ProductListRDD.values()

def extractPairsOfProducts(transaction):

    products = list(transaction)

    returnedPairs = []
    
    for product1 in products:
        for product2 in products:
            if product1<product2:
                returnedPairs.append(((product1, product2), 1))
                
    return returnedPairs

PairsOfProductsRDD = ProductID_ValuesRDD.flatMap(extractPairsOfProducts)
PairsFrequenciesRDD = PairsOfProductsRDD.reduceByKey(lambda count1, count2: count1 + count2)
atLeast2PairsFrequenciesRDD = PairsFrequenciesRDD.filter(lambda inputTuple: inputTuple[1]>1)
atLeast2PairsFrequenciesSortedRDD = atLeast2PairsFrequenciesRDD.sortBy(lambda inputTuple: inputTuple[1], False)
atLeast2PairsFrequenciesSortedRDD.saveAsTextFile(outputPath)